# Spopularity

---


# Requesting new token

First request is for updating the token, if needed. Credentials are stored in `credentials.json` -file.


In [1]:
import json
import spotipy
import time


def get_token():
    client_id = ""
    client_secret = ""
    token = ""
    expiration = 0

    with open("credentials.json", "r") as f:
        credentials = json.load(f)
        client_id = credentials["client_id"]
        client_secret = credentials["client_secret"]
        token = credentials["token"]
        expiration = credentials["expiration"]

    if not token or expiration < time.time():
        token = spotipy.oauth2.SpotifyClientCredentials(
            client_id=client_id, client_secret=client_secret
        )
        token = token.get_access_token(as_dict=False)
        credentials["token"] = token
        credentials["expiration"] = int(time.time() + 3600)

        with open("credentials.json", "w") as f:
            f.write(json.dumps(credentials))

    return token


s = spotipy.Spotify(get_token())

In [2]:
import pandas as pd


billboard_top_5 = [
    "1kuGVB7EU95pJObxwvfwKS",
    "2IGMVunIBsBLtEQyoI1Mu7",
    "4KULAymBBJcPRpk1yO4dOG",
    "1Lo0QY9cvc8sUB2vnIOxDT",
    "1BxfuPKGuaTgP7aM0Bbdwr",
]

track_details = s.tracks(billboard_top_5)

tracks = []
for t in track_details["tracks"]:
    track_name = t["name"]
    track_id = t["id"]
    artist_name = t["artists"][0]["name"]
    popularity = t["popularity"]
    tracks.append([track_name, track_id, artist_name, popularity])

df = pd.DataFrame(tracks, columns=["Track Name", "Track ID", "Artist Name", "Popularity"])

print(df.head())
print(df.describe())

                                      Track Name                Track ID  \
0                                        vampire  1kuGVB7EU95pJObxwvfwKS   
1                             Paint The Town Red  2IGMVunIBsBLtEQyoI1Mu7   
2  I Remember Everything (feat. Kacey Musgraves)  4KULAymBBJcPRpk1yO4dOG   
3                                       Fast Car  1Lo0QY9cvc8sUB2vnIOxDT   
4                                   Cruel Summer  1BxfuPKGuaTgP7aM0Bbdwr   

      Artist Name  Popularity  
0  Olivia Rodrigo          92  
1        Doja Cat         100  
2      Zach Bryan          91  
3      Luke Combs          90  
4    Taylor Swift          98  
       Popularity
count    5.000000
mean    94.200000
std      4.494441
min     90.000000
25%     91.000000
50%     92.000000
75%     98.000000
max    100.000000
